In [1]:
from sympy.physics.quantum import Commutator, Operator
import sympy
from sympy import *
from IPython.display import display
import numpy as np

# Hamiltonian

Define operators and constants

In [2]:
U, F,G, delta_a, delta_b, J, kappa_a, kappa_ad,eta = symbols(r'U F G \Delta_a \Delta_b J \kappa_{a} \kappa_{ad} \eta')

a = Symbol(r'a', commutative=False)
ad = Symbol(r'a^\dag', commutative=False)
b = Symbol(r'b', commutative=False)
bd = Symbol(r'b^\dag', commutative=False)

rho = Symbol(r'\rho', commutative=False)

Construct Hamiltonian

In [3]:
Ham1 = J*(ad*b+a*bd) + F*(ad+a) -delta_a*ad*a -delta_b*bd*b  + Rational(1, 2)*U*ad*ad*a*a
Ham2 = -delta_a*ad*a -delta_b*bd*b
Ham3 = Mul(U,ad,ad,a,a,evaluate=False)
Ham4 = UnevaluatedExpr(Mul(Rational(1, 2)*U,ad,ad,a,a,evaluate=False))
Ham5 = J*(a*bd+ad*b)
Ham6 = G*(ad*ad+a*a)

MyHam = Ham6
display(MyHam)

G*(a**2 + a^\dag**2)

Jump operators

In [4]:

def getJump(constant,c,rho):
    cd = Dagger(c)
    return UnevaluatedExpr(constant*(cd*rho*c - Rational(1, 2)*(c*cd*rho + rho*c*cd)))

D1 = (kappa_a/2)*(a*rho*ad - Rational(1, 2)*(ad*a*rho + rho*ad*a)) 
D1ad = (kappa_ad/2)*(ad*rho*a - Rational(1, 2)*(a*ad*rho + rho*a*ad))
D2 = eta*(a*a*rho*ad*ad - Rational(1, 2)*(ad*ad*a*a*rho + rho*ad*ad*a*a))
#D1 = getJump(kappa_a,a,rho) !!! Not working because the Dagger() is not recognized as one of the defined symbols when applying the operators!
#display(D1)

    

# Lindblad master equation

In [5]:
#ME_comm = -sympy.I* (Commutator(MyHam, rho))
ME_comm = D1ad
display(ME_comm)
ME= ME_comm.doit()
#display(ME)
ME = expand(ME)
display(ME)

\kappa_{ad}*(a^\dag*\rho*a - (\rho*a*a^\dag + a*a^\dag*\rho)/2)/2

-\kappa_{ad}*\rho*a*a^\dag/4 - \kappa_{ad}*a*a^\dag*\rho/4 + \kappa_{ad}*a^\dag*\rho*a/2

# Convert to desired phase space function

In [6]:
from phasespaceconversion import pow_to_mul_sep2,PhaseSpaceFunction

In [7]:
clean_term = ad*rho*a
display(clean_term)
clean_termNoRho =  clean_term.subs(rho,1 )
print(clean_term.args)

def getPosRho(clean_term,rho):
    counter=0
    items= clean_term.args
    n_items = len(items)
    print(items)
    for k in range(n_items):
        item = items[k]
        if item==rho:
           break    # break here
        else:
            [op, n] = item.as_base_exp()
            counter = counter+n
            print(counter)

    print('rho occurs on position  {}'.format(counter))
    return counter

def labelOperators(ordered_operators, pos_rho):
    n_operators = len(ordered_operators)
    operator_labels = ['RHS']*n_operators
    operator_labels[:pos_rho]=['LHS']*pos_rho
    return operator_labels

pos_rho=getPosRho(clean_term,rho)
ordered_operators = pow_to_mul_sep2(clean_termNoRho)

operator_labels=labelOperators(ordered_operators, pos_rho)

print(ordered_operators)
print(operator_labels)
operator_labels =[] # label each individual operator in the list as RHS or LHS of Rho
print(clean_termNoRho.args)

a^\dag*\rho*a

(a^\dag, \rho, a)
(a^\dag, \rho, a)
1
rho occurs on position  1
[a^\dag, a]
['LHS', 'RHS']
(a^\dag, a)


In [8]:
# ordered_operators = pow_to_mul_sep2(clean_termNoRho)
# display(ordered_operators)



Wigner

In [9]:
myWigner = PhaseSpaceFunction(ME,'W')
W_eqn = myWigner.getFPfromME()
print('Result')
display(expand(W_eqn))

(\kappa_{ad}*a^\dag*\rho*a/2,
 -\kappa_{ad}*\rho*a*a^\dag/4,
 -\kappa_{ad}*a*a^\dag*\rho/4)

\kappa_{ad}*a^\dag*\rho*a/2

a^\dag*\rho*a

a^\dag*a

(a^\dag, \rho, a)
1
rho occurs on position  1
j=
0
Term 0 :
ops left in mid term, order operators applied:
[a^\dag]


...

a*a^\dag

Term 2 :
LHS term, order operators applied:
[a^\dag, a]


-\kappa_{ad}*(\alpha*(\alpha^**W(\alpha, \alpha^*, \beta, \beta^*) - Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha)/2) + \alpha^**Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/2 + W(\alpha, \alpha^*, \beta, \beta^*)/2 - Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)/4)/4

Result


-\alpha*\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha)/4 - \alpha^**\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/4 - \kappa_{ad}*W(\alpha, \alpha^*, \beta, \beta^*)/2 + \kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)/4

HusimiQ

In [10]:
myQ= PhaseSpaceFunction(ME,'Q')
Q_eqn = myQ.getFPfromME()
print('Result')
display(expand(Q_eqn))

(\kappa_{ad}*a^\dag*\rho*a/2,
 -\kappa_{ad}*\rho*a*a^\dag/4,
 -\kappa_{ad}*a*a^\dag*\rho/4)

\kappa_{ad}*a^\dag*\rho*a/2

a^\dag*\rho*a

a^\dag*a

(a^\dag, \rho, a)
1
rho occurs on position  1
j=
0
Term 0 :
ops left in mid term, order operators applied:
[a^\dag]


...

a*a^\dag

Term 2 :
LHS term, order operators applied:
[a^\dag, a]


-\kappa_{ad}*(\alpha*\alpha^**Q(\alpha, \alpha^*, \beta, \beta^*) + \alpha^**Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + Q(\alpha, \alpha^*, \beta, \beta^*))/4

Result


-\alpha*\kappa_{ad}*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha)/4 - \alpha^**\kappa_{ad}*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/4 - \kappa_{ad}*Q(\alpha, \alpha^*, \beta, \beta^*)/2

P representation

In [11]:
myP= PhaseSpaceFunction(ME,'P')
P_eqn = myP.getFPfromME()
print('Result')
display(expand(P_eqn))

(\kappa_{ad}*a^\dag*\rho*a/2,
 -\kappa_{ad}*\rho*a*a^\dag/4,
 -\kappa_{ad}*a*a^\dag*\rho/4)

\kappa_{ad}*a^\dag*\rho*a/2

a^\dag*\rho*a

a^\dag*a

(a^\dag, \rho, a)
1
rho occurs on position  1
j=
0
Term 0 :
ops left in mid term, order operators applied:
[a^\dag]


...

a*a^\dag

Term 2 :
LHS term, order operators applied:
[a^\dag, a]


-\alpha*\kappa_{ad}*(\alpha^**P(\alpha, \alpha^*, \beta, \beta^*) - Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha))/4

Result


-\alpha*\kappa_{ad}*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha)/4 - \alpha^**\kappa_{ad}*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/4 - \kappa_{ad}*P(\alpha, \alpha^*, \beta, \beta^*)/2 + \kappa_{ad}*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)/2

# Separate Real and imaginary Parts Wigner

## Not working for now, use mathematica file instead

In [12]:
W_exp_comp = expand(W_eqn)
display(W_exp_comp)

# redefine symbols in original equation (not so nice that this has to be repeated but okay)

#field = Function('W')(alpha,self.alphas,self.beta,self.betas) # can be W, Q or P
alpha = Symbol(r'\alpha', commutative=True)
alphas = Symbol(r'\alpha^*', commutative=True)
beta = Symbol(r'\beta', commutative=True)
betas = Symbol(r'\beta^*', commutative=True)

# define real and imag parts:
alpha_r = Symbol(r'a_R', commutative=True)
alpha_i = Symbol(r'a_I', commutative=True)
beta_r = Symbol(r'b_R', commutative=True)
beta_i = Symbol(r'b_I', commutative=True)

# substitute in expression
W_exp_comp_sep =W_exp_comp.subs(alpha, alpha_r+sympy.I*alpha_i )
W_exp_comp_sep =W_exp_comp.subs(alphas, alpha_r-sympy.I*alpha_i )
W_exp_comp_sep = W_exp_comp.subs(beta, beta_r+sympy.I*beta_i )
W_exp_comp_sep = W_exp_comp.subs(betas, beta_r-sympy.I*beta_i )

display(W_exp_comp_sep)
W_exp_Re = re(W_exp_comp)
display(W_exp_Re)

-\alpha*\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha)/4 - \alpha^**\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/4 - \kappa_{ad}*W(\alpha, \alpha^*, \beta, \beta^*)/2 + \kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)/4

-\alpha*\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha)/4 - \alpha^**\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha^*)/4 - \kappa_{ad}*W(\alpha, \alpha^*, \beta, -I*b_I + b_R)/2 + \kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha, \alpha^*)/4

-re(\kappa_{ad}*W(\alpha, \alpha^*, \beta, \beta^*))/2 + re(\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*))/4 - re(\alpha*\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha))/4 - re(\alpha^**\kappa_{ad}*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*))/4

In [13]:
W_exp_comp = expand(W_eqn)
W_exp_comp.atoms()
W_exp_comp.atoms(Function)

{W(\alpha, \alpha^*, \beta, \beta^*)}

In [14]:
latex(W_exp_comp)

'- \\frac{\\alpha \\kappa_{ad} \\frac{\\partial}{\\partial \\alpha} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)}}{4} - \\frac{\\alpha^{*} \\kappa_{ad} \\frac{\\partial}{\\partial \\alpha^{*}} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)}}{4} - \\frac{\\kappa_{ad} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)}}{2} + \\frac{\\kappa_{ad} \\frac{\\partial^{2}}{\\partial \\alpha^{*}\\partial \\alpha} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)}}{4}'